File: ratings.ipynb\
Author: alexkobz\
Date: 07.05.2024\
Task: DSRFU-275

#### Шаг 0. Импорт и получение токена

In [1]:
import asyncio
import requests
import pandas as pd
import numpy as np
import nest_asyncio; nest_asyncio.apply()
from datetime import datetime as dt, date, timedelta
from functions.divide_chunks import divide_chunks
from math import ceil
from functions.get_date import last_day_month, last_day_month_str, last_work_date_month, last_work_date_month_str

# последний день месяца
REPORT_DATE = last_day_month
REPORT_DATE_STR = last_day_month_str
# увеличиваем количество столбцов и строк для просмотра
pd.set_option('display.max_rows', 200, 'display.max_columns', 200)

In [2]:
from rudata.RuDataDF import RuDataDF
from rudata.RuDataRequest import RuDataRequest
RuDataRequest.set_headers()

Getting token started
Getting token finished


#### Шаг 1. Загрузка stage таблиц 

[FintoolReferenceData](https://docs.efir-net.ru/dh2/#/Info/FintoolReferenceData?id=post-fintoolreferencedata) - Получить расширенный справочник по финансовым инструментам.

In [3]:
FintoolReferenceData = RuDataDF("FintoolReferenceData").df
FintoolReferenceData.head()

FintoolReferenceData started
post start
post start
post start
post start
post start
post processing
post processing
post processing
post processing
post processing
1500, 0
post start
post start
post start
post start
post start
post processing
post processing
post processing
post processing
post processing
1500, 1500
post start
post start
post start
post start
post start
post processing
post processing
post processing
post processing
post processing
1500, 3000
post start
post start
post start
post start
post start
post processing
post processing
post processing
post processing
post processing
1500, 4500
post start
post start
post start
post start
post start
post processing
post processing
post processing
post processing
post processing
1500, 6000
post start
post start
post start
post start
post start
post processing
post processing
post processing
post processing
post processing
1500, 7500
post start
post start
post start
post start
post start
post processing
post processing
post proces

,facevalue_m,faceftname_m,sumissuevol_m,summarketvol_m,numcoupons_m,nrdcode,regcode_m,regdate_m,regdistdate_m,begdistdate_m,enddistdate_m,endmtydate_m,daysall_m,fullname_nrd,rpregorg_nrd,rpstate_nrd,issuenumber,currentfacevalue_nrd,isqualified_nrd,bondseries,islombardcbr_nrd,numcoupons_nrd,facevalue_nrd,faceftname_nrd,summarketvol_nrd,regcode_nrd,regdate_nrd,regorg_nrd,begdistdate_nrd,daysall_nrd,enddistdate_nrd,endmtydate_nrd,regdistdate_nrd,sumissuevol_nrd,drqty,isincodebase_nrd,nrdcompcode,shqty,summarketvolorig_nrd,issuername_nrd,planmtydate_nrd,firstcoupondate_nrd,depositary_nrd,registrar_nrd,placeprice_nrd,micexlist_nrd,minlot_nrd,formarketbonds_nrd,registraracctype_nrd,registraracctypedate_nrd,facialacc_nrd,acc_open_date_nrd,fraction_nrd,specialized_depo_nrd,cmp_code_nsd_issuer_nrd,cmp_code_nsd_managing_nrd,ratetypenamerus_nrd,ratetypenameeng_nrd,coupontypename_nrd,base_month,base_year,basis_nrd,sectypenamerus_nrd,sectypenameeng_nrd,secformnamerus_nrd,secformnameeng_nrd,sectypenamebr_nrd,secstaterus_nrd,secstateeng_nrd,placementtype_nrd,share_category_name_nrd,share_category_name_en_nrd,dr_category_name_nrd,dr_category_name_en_nrd,company_state_name_nrd,rate_type_id,cp_type_id,coupon_period_base_id,sec_type_id,sec_form_id,sec_state_id,placement_type_id,share_category_id,dr_category_mn,company_state_mn,sector4212u,sna2008,fullname_en_nrd,seniorityname,fintoolid,moex_code,moex_name,fullname,isin144a,isincode,nickname,progfintoolid,regcode,regdate,regorg,amortisedmty,basis,bondstructuralpar,cfi,coupontype,fintooltype,guaranteeamount,guaranteetype,haveoffer,isconvertible,isguaranteed,maturitygroup,securitization,accruedintcalctype,country,couponperyear,faceftname,facevalue,guarantval,havedefault,numcoupons,status,sumissueval,sumissuevol,summarketval,summarketvol,borrowername,borrowerokpo,borrowersector,borroweruid,borrowerinn,borrowercountry,borrowerregionname,issuername,issuerokpo,issuersector,issueruid,issuerinn,issuercountry,issuerregionname,assetssector,numguarantors,begdistdate,begmtydate,daysall,enddistdate,endmtydate,privatedist,regdistdate,securitytype,securitykind,nominaltype,update_time,isregion,nsd_service,raterevisionsperyear,floatratename,issubordinated,note,listing_level,havecovenant,haverepayment,haveindexedfv,numdecimals,couponperiod,ismatched,firstcoupondate,couponrate,iscomplicated,available_for_unqualified,sec_type_br_code,original_country,qualified_test_id,qualified_test_id_tm,base_asset_fintoolid,isqualified,listing_level_spb,have_mm_moex,have_mm_spb,beg_book_date,seniority,tradesite,current_coupontype,profit_type,counter,rn
0,NaN,NaN,NaN,NaN,NaN,RU0002154147,3.0,3.0,NaN,NaN,NaN,4.0,NaN,"ОПИФ рыночных финансовых инструментов ""ДАКО""",None,None,None,NaN,0.0,None,NaN,NaN,NaN,None,NaN,0076-59774694,2002-12-24T00:00:00,ФКЦБ России,None,NaN,None,2018-01-14T00:00:00,None,NaN,NaN,None,MC0233700000,NaN,NaN,"АО ""РЕГИОН ЭсМ""",None,None,None,MC0162600000,NaN,None,NaN,NaN,НДЦД,2013-10-22T00:00:00,001800000013,2003-03-18T00:00:00,5.0,MC0162600000,PIF1917,None,None,None,None,None,None,None,инвестиционные паи открытого паевого инвестици...,None,Бездокументарная именная,uncertificated registered,Паи (акции) инвестиционных фондов-резидентов,None,None,None,None,None,None,None,Прекращен,NaN,NaN,NaN,4.0,2.0,3.0,NaN,NaN,None,C,None,S125,DAKO UNIT,None,6813,RU0002154147,РЕГ ИП АиО,Открытый паевой инвестиционный фонд рыночных ф...,None,RU0002154147,РегионЭМ-ДАКО ПИФ,NaN,0076-59774694,2002-12-24T00:00:00,ФКЦБ,NaN,None,None,CIOXLU,None,Фонд,None,None,NaN,NaN,NaN,None,None,NaN,RU,NaN,RUB,NaN,NaN,NaN,NaN,Аннулирован,5.000000e+07,NaN,NaN,NaN,None,None,None,NaN,None,None,None,Регион ЭсМ,58222277,Финанс.сервисы,87190.0,7730149408,RU,г.Москва,None,NaN,2003-01-14T00:00:00,None,NaN,2003-01-27T00:00:00,2020-03-27T00:00:00,NaN,2003-01-31T00:00:00,Открытый,РыночФинансИструментов,None,2024-04-27T19:25:18,NaN,Снят с обслуживания,NaN,None,NaN,Основание прекращения Фонда и дата его возникн...,NaN,None,None,None,NaN,NaN,None,None,NaN,None,F

[SecurityRatingTable](https://docs.efir-net.ru/dh2/#/Rating/SecurityRatingTable?id=post-securityratingtable) - Получить рейтинги нескольких бумаг и связанных с ними компаний на заданную дату.

In [4]:
%%time
SecurityRatingTable = RuDataDF("SecurityRatingTable").df
SecurityRatingTable.head()

SecurityRatingTable started
post start
post start
post start
post start
post start
post processing
post processing
post processing
post processing
post processing
1313, 0
post start
post start
post start
post start
post start
post processing
post processing
post processing
post processing
post processing
754, 1313
post start
post start
post start
post start
post start
post processing
post processing
post processing
post processing
post processing
1693, 2067
post start
post start
post start
post start
post start
post processing
post processing
post processing
post processing
post processing
1236, 3760
post start
post start
post start
post start
post start
post processing
post processing
post processing
post processing
post processing
1303, 4996
post start
post start
post start
post start
post start
post processing
post processing
post processing
post processing
post processing
1267, 6299
post start
post start
post start
post start
post start
post processing
post processing
post processi

,id,hid,code_name,id_value,forecast,change,date_award,role,okpo,fininstid,fintoolid,isin,id_rating,code,shortname_rus
0,0,6,MIRA-I-Corp-LT-Nsc,Снят,None,снят,2016-03-18T00:00:00,1,None,7319.0,NaN,None,116,None,Татнефть
1,0,6,MIRA-I-Corp-LT-Nsc,Снят,None,снят,2016-03-18T00:00:00,1,None,43888.0,NaN,None,116,None,Уралкалий
2,0,123,MIRA-I-Cr&D-LT-Nsc,Снят,None,снят,2015-02-25T00:00:00,1,None,43888.0,NaN,None,32,None,Уралкалий
3,0,133,MIRA-B-Cr&D-LT-Nsc,Снят,None,снят,2016-03-18T00:00:00,1,None,7205.0,NaN,None,117,None,Сбербанк России
4,0,133,MIRA-B-Cr&D-LT-Nsc,Снят,None,снят,2016-03-18T00:00:00,1,None,6455.0,NaN,None,117,None,Банк Возрождение


[ExchangeTree](https://docs.efir-net.ru/dh2/#/Info/ExchangeTree?id=post-exchangetree) - Получить иерархию торговых площадок/источников, используемых Интерфакс

In [3]:
ExchangeTree = RuDataDF("ExchangeTree").df
ExchangeTree.head()

ExchangeTree started
ExchangeTree finished. ExchangeTree shape (239, 11)


,id,id_parent,shortname_rus,shortname_eng,fullname_rus,fullname_eng,sort_order,update_date,counter,rn,report_monthyear
0,144,63,Нал,Cash,Наличная валюта,Cash,0,2020-12-28T17:03:09,239,1,122024
1,270,63,RUDIP 2,RUDIP 2,RUDIP 2,RUDIP 2,1,2022-10-31T19:41:24,239,2,122024
2,84,83,США,US,Макроиндикаторы США,USA macroindicators,1,2020-12-28T17:03:09,239,3,122024
3,85,83,РФ,RU,Макроиндикаторы Россия,Russia macroindicators,2,2020-12-28T17:03:09,239,4,122024
4,2,1,РТС,RTS,РТС индексы,RTS indexes,2,2020-12-28T17:03:09,239,5,122024


[ListScaleValues](https://docs.efir-net.ru/dh2/#/Rating/ListScaleValues?id=post-listscalevalues) - Список шкал значений рейтингов

In [4]:
ListScaleValues = RuDataDF("ListScaleValues").df
ListScaleValues.head()

ListScaleValues started
ListScaleValues finished. ListScaleValues shape (2019, 9)


,id,scale_id,rating_value,rating_level,fullname_rus,description,aggregation_type,aggregation_level,report_monthyear
0,1,1,AAA(rus),1,Наивысший уровень кредитоспособности,Национальные рейтинги категории «ААА» обознача...,None,NaN,122024
1,2,1,AA+(rus),2,None,None,None,NaN,122024
2,3,1,AA(rus),3,Очень высокая кредитоспособность,Национальные рейтинги категории «AA» обозначаю...,None,NaN,122024
3,4,1,AA-(rus),4,None,None,None,NaN,122024
4,5,1,A+(rus),5,None,None,None,NaN,122024


[ListRatings](https://docs.efir-net.ru/dh2/#/Dictionary/ListRatings?id=post-listratings) - Список рейтингов

In [5]:
ListRatings = RuDataDF("ListRatings").df
ListRatings.head()

ListRatings started
ListRatings finished. ListRatings shape (362, 26)


,id,rating_id,code_name,agency,fullname_rus,fullname_eng,for_instrument,for_company,agency_eng,official_name,is_credit,term_type,term_type_name,currency_type,currency_type_name,scale_type,scale_type_name,is_archive,is_accred_br,doc_511p,doc_421p,doc_180i,agency_id,usedInAgg,scale_id,aggregation_type
0,134,104,MDS-B-Cr&D-LT-KZsc,Мудиз,"Рейтинг банковских депозитов, долгосрочный, Ка...","Bank deposit ratings, long-term, kazakhstan scale",0,1,Moody's,LT Bank Deposits (kaz),1,L,Долгосрочный,L,Локальная,N,Национальная,0,1.0,0,0,0,2,False,22,None
1,135,109,MDS-B-Cr&D-LT-UAsc,Мудиз,"Рейтинг банковских депозитов, долгосрочный, Ук...","Bank deposit ratings, long-term, ukrainian scale",0,1,Moody's,LT Bank Deposits (ukr),1,L,Долгосрочный,L,Локальная,N,Национальная,0,1.0,0,0,0,2,False,23,None
2,451,110,RUS-AM-FS&R-Isc,РусРейтинг,"Рейтинг надежности управляющих компаний, между...","Reliability rating of the management company, ...",0,1,RusRating,Рейтинги надежности управляющих компаний,0,None,None,None,None,I,Международная,1,NaN,0,0,0,6,False,35,None
3,452,111,RUS-AM-FS&R-Nsc,РусРейтинг,"Рейтинг надежности управляющих компаний, нацио...","Reliability rating of the management company, ...",0,1,RusRating,Рейтинги надежности управляющих компаний,0,None,None,L,Локальная,N,Национальная,1,NaN,0,0,0,6,False,36,None
4,446,114,EXP-IN-FS&R-Isc,Эксперт РА,"Рейтинги надежности страховых компаний, междун...","Reliability rating of insurance companies, int...",0,1,Expert RA,Рейтинги финансовой надежности страховых компаний,1,None,None,None,None,I,Международная,1,1.0,0,0,0,4,False,47,None


[Emitents](https://docs.efir-net.ru/dh2/#/Info/Emitents?id=post-emitents) - Получить краткий справочник по эмитентам.

In [8]:
Emitents = RuDataDF("Emitents").df
Emitents.head()

Emitents started
post start
post start
post start
post start
post start
post processing
post processing
post processing
post processing
post processing
1500, 0
post start
post start
post start
post start
post start
post processing
post processing
post processing
post processing
post processing
1500, 1500
post start
post start
post start
post start
post start
post processing
post processing
post processing
post processing
post processing
1500, 3000
post start
post start
post start
post start
post start
post processing
post processing
post processing
post processing
post processing
1500, 4500
post start
post start
post start
post start
post start
post processing
post processing
post processing
post processing
post processing
1500, 6000
post start
post start
post start
post start
post start
post processing
post processing
post processing
post processing
post processing
1500, 7500
post start
post start
post start
post start
post start
post processing
post processing
post processing
post pr

,id_emitent,fininstid,shortname_rus,shortname_eng,inn,okpo,ogrn,code,sector,legal_address,phone,www,update_date,fullname_rus_nrd,fullname_eng_nrd,shortname_rus_nrd,shortname_eng_nrd,company_type_short_name,state_reg_num,state_reg_date,state_reg_name,egrul_date,egrul_organ_name,country,lei_code,bik,fax,e_mail,credit_cmp,is_bank_4_non_resident,swift,country_oksm,country_oksm_cbr,country_name_rus,country_name_eng,region_inn,region_soato,region_name,fullname_rus,sparkid,br_fcsm_reg_code,capital,capital_unit,okved,oecd_lvl,oktmo,okato,post_address,okopf,okogu,have_rating,have_risk,is_finorg,is_control_by_finorg,is_gov_or_cb,on_behalf_of_state,is_subjectrf,is_cis_reg,issuer_nrd,tin,kpp,state_reg_number,market_id_nrd,market_name_rus_nrd,market_name_eng_nrd,sp_rx_entity_id,sp_shortname,mds_shortname,mds_org_id,fch_id,fch_shortname,is_monopoly,is_strategic,ifo_list,isregion,sic,sector4212u,sna2008,reg_code,reg_date,reg_org,note,primary_reg_date,inn_fle,other_tin,other_tin_name,okfs_id,okfs_name,is_branch,counter,rn
0,85020,7435,Семь дней Издательство,None,7703038770,42725020,1027700497109,XX026540,None,"103104, г. Москва, Тверской бульвар, д. 9, стр.1",(495) 753-41-41,http://www.7days.ru/,2022-11-01T12:38:52.452305,None,None,None,None,None,None,None,None,None,None,RUS,None,None,None,marketing@7days.ru,None,None,None,643.0,643.0,РОССИЯ,Russia,77,45,г.Москва,"""Издательство Семь Дней"", ЗАО",NaN,None,1.500000e+07,RUB,None,7,None,None,"125080, г. Москва, Ленинградское шоссе, д.5а",None,None,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,None,NaN,None,None,None,None,None,None,None,None,0,0,None,0.0,None,None,None,1027700497109,2002-12-05T00:00:00,Управление Министерства Российской Федерации п...,"Издания: Итоги, Семь Дней ТВ-Программа, Карава...",1995-10-24T00:00:00,None,None,ДопИНН,None,None,False,40731,601
1,26466,7441,Финпромбанк АКБ,AKB FINPROMBANK,7707077586,17508352,1027739174759,FPBN,Банки,"107045, г. Москва, пер. Последний, д. 24",(495)7300333; (495)7301450,www.fpb.ru,2024-04-06T15:52:50,"Акционерный Коммерческий Банк ""Финансово-Промы...",None,"АКБ ""ФИНПРОМБАНК"" (ПАО)",FINPROMBANK A/O,ПАО,2157,1992-11-19T00:00:00,Банк России,2002-09-11T00:00:00,Межрайонная инспекция МНС России № 39 по г. Мо...,RUS,2534006BFFXTH9Q8EL76,044525549,(495) 730-14-50,post@finprombank.ru,1,0,None,643.0,643.0,РОССИЯ,Russia,77,45,г.Москва,"Акционерный коммерческий банк ""Финансово-Промы...",1469919.0,02157-B,2.185400e+09,RUB,64.19,7,45379000000,45286570000,"107045, г. Москва, пер. Последний, д. 24",12247,1500010,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,MC0124400000,None,770201001,2157,2.0,Корпоративный,corporate,None,None,Finprombank,823257069,None,None,0,0,None,0.0,6020,None,S122,1027739174759,2002-09-11T00:00:00,None,None,1992-11-19T00:00:00,None,None,ДопИНН,16,Частная собственность,False,40731,602
2,30456,7447,Роскосмосбанк,None,7727051787,29351619,1027739224611,FSEB,Банки,"125196,г. Москва, ул. Бутырский Вал, д. 18 стр. 2",None,None,2023-08-01T14:45:10.56341,None,None,None,None,НАО,None,None,None,None,None,RUS,253400QHJ4G71X970104,044525904,None,None,None,None,None,643.0,643.0,РОССИЯ,Russia,77,45,г.Москва,"""РОСКОСМОСБАНК"", АО",1473158.0,02989-B,4.191000e+09,RUB,64.19,7,45382000000,45286585000,"125196,г. Москва, ул. Бутырский Вал, д. 18 стр. 2",12267,1500010,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,None,None,771001001,None,NaN,None,None,None,None,Fundservicebank,808812986,None,None,0,0,None,0.0,6020,None,S122,1027739224611,2002-09-20T00:00:00,None,Прекращением деятельности АО «РОСКОСМОСБАНК» в...,1994-08-03T00:00:00,None,None,ДопИНН,16,Частная собственность,False,40731,603
3,34893,7448,ФорБанк КБ,None,2202000656,21417572,1022200526061,XX004550,Банки,"105066,г. Москва, ул. Новая Басманная, д. 29 с...",(385)2231547; (385)2232474; (385)2237959,www.forbank.alt.ru,2023-08-01T14:45:10.563411,None,None,None,None,НАО,None,None,None,None,None,RUS,2534007PP13M5HKBPA86,044525074,None,forbank@forbank.alt.ru,None,None,None,643.0,643.0,РОССИЯ,Russia,77,45,г.Москва,"Коммерческий бан

[OfferorsGuarants](https://docs.efir-net.ru/dh2/#/Bond/OfferorsGuarants?id=post-offerorsguarants) - Возвращает список гарантов/оферентов для инструмента

In [9]:
OfferorsGuarants = RuDataDF("OfferorsGuarants").df
OfferorsGuarants.head()

OfferorsGuarants started
post start
post start
post start
post start
post start
post processing
post processing
post processing
post processing
post processing
500, 0
post start
post start
post start
post start
post start
post processing
post processing
post processing
post processing
post processing
500, 500
post start
post start
post start
post start
post start
post processing
post processing
post processing
post processing
post processing
500, 1000
post start
post start
post start
post start
post start
post processing
post processing
post processing
post processing
post processing
500, 1500
post start
post start
post start
post start
post start
post processing
post processing
post processing
post processing
post processing
500, 2000
post start
post start
post start
post start
post start
post processing
post processing
post processing
post processing
post processing
500, 2500
post start
post start
post start
post start
post start
post processing
post processing
post processing
post p

,fintoolId,fininstid_guarant,fininstid_offeror,counter,rn
0,132717,6120,NaN,5243,301
1,132803,6936,NaN,5243,302
2,132804,102883,NaN,5243,303
3,132809,102676,NaN,5243,304
4,132813,93913,NaN,5243,305


[CompanyRatingsTable](https://docs.efir-net.ru/dh2/#/Rating/CompanyRatingsTable?id=post-companyratingstable) - Получить рейтинги нескольких компаний на заданную дату.

In [10]:
CompanyRatingsTable = RuDataDF("CompanyRatingsTable").df
CompanyRatingsTable.head()

CompanyRatingsTable started
post start
post start
post start
post start
post start
post processing
post processing
post processing
post processing
post processing
299, 0
post start
post start
post start
post start
post start
post processing
post processing
post processing
post processing
post processing
534, 299
post start
post start
post start
post start
post start
post processing
post processing
post processing
post processing
post processing
825, 833
post start
post start
post start
post start
post start
post processing
post processing
post processing
post processing
post processing
760, 1658
post start
post start
post start
post start
post start
post processing
post processing
post processing
post processing
post processing
515, 2418
post start
post start
post start
post start
post start
post processing
post processing
post processing
post processing
post processing
501, 2933
post start
post start
post start
post start
post start
post processing
post processing
post processing
post

,name_ra,dt,last,last_dt,change,forecast,advanced,prev,prev_dt,id_rating,code,code_type,company_name
0,Рейтинг подтвержден,2024-08-27T00:00:00,ruBBB,2020-10-01T00:00:00,Подтвержден,Стабильный,Основной,None,None,161,50904,FININSTID,Татхимфармпрепараты
1,Рейтинг отозван,2017-04-21T00:00:00,Снят,2017-04-21T00:00:00,снят,None,Основной,A (I),2016-05-18T00:00:00,161,62447,FININSTID,Якутуглестрой
2,Рейтинг отозван,2017-04-21T00:00:00,Снят,2017-04-21T00:00:00,снят,None,Основной,B13.ru,2016-05-18T00:00:00,902,62447,FININSTID,Якутуглестрой
3,Рейтинг отозван,2017-04-21T00:00:00,Снят,2017-04-21T00:00:00,снят,None,Основной,B13.ru,2016-05-18T00:00:00,904,62447,FININSTID,Якутуглестрой
4,Рейтинг отозван,2016-03-18T00:00:00,Снят,2016-03-18T00:00:00,снят,None,Основной,Baa1.ru,2015-12-22T00:00:00,6,43589,FININSTID,Кокс


#### Шаг 2. Предобработка таблиц до экспорта в excel

In [11]:
#добавляем информацию из справочника в нашу таблицу с компаниями
ExtendedSecurityRatingTable = SecurityRatingTable.merge(ListRatings[['rating_id','agency','official_name','term_type_name','scale_type_name','scale_id']], how='left', left_on='id_rating', right_on='rating_id')
ExtendedSecurityRatingTable = ExtendedSecurityRatingTable.merge(ListScaleValues, how='left', left_on=['scale_id', 'id_value'], right_on=['scale_id', 'rating_value'])
ExtendedSecurityRatingTable = ExtendedSecurityRatingTable[[
    'id_x', 'hid', 'code_name', 'id_value', 'forecast', 'change',
    'date_award', 'role', 'okpo', 'fininstid', 'fintoolid', 'isin',
    'id_rating', 'code', 'shortname_rus', 'rating_id', 'agency',
    'official_name', 'term_type_name', 'scale_type_name', 'scale_id',
    'aggregation_level', 'aggregation_type']].rename(
    columns={
    'id_x': 'id', 'aggregation_level': 'level', 'aggregation_type': 'agg_type'
    }
)
agencies = ['АКРА', 'Эксперт РА', 'НКР', 'НРА'
           # ]
            , 'Мудис Интерфакс']
ExtendedSecurityRatingTable = ExtendedSecurityRatingTable[ExtendedSecurityRatingTable['agency'].isin(agencies)]
ExtendedSecurityRatingTable = ExtendedSecurityRatingTable[ExtendedSecurityRatingTable['term_type_name'] != 'Краткосрочный']
ExtendedSecurityRatingTable = ExtendedSecurityRatingTable[~ExtendedSecurityRatingTable['id_value'].isin(['****','Приостановлен','Снят'])]
ExtendedSecurityRatingTable = ExtendedSecurityRatingTable[~ExtendedSecurityRatingTable['isin'].isna()]


In [12]:
akra_list = [
'Structured Finance Instrument Rating',
 'Рейтинг эмиссии, международная шкала',
 'Старший необеспеченный рейтинг эмиссии',
 'Старший обеспеченный рейтинг эмиссии',
 'Субординированный рейтинг эмиссии'
]
expert_list = [
    'Structured finance instrument rating',
    'Кредитный рейтинг долговых инструментов'
]
nkr_list = [
   # 'Кредитный рейтинг долговых обязательств'
]
nra_list = [
 #   'Кредитный рейтинг отдельных выпусков облигаций',
  #  'Рейтинг кредитоспособности выпусков облигаций Индивидуальный'
]
moodys_list = [
    'NSR Senior Unsecured (Domestic)',
 'Senior Unsecured (Domestic); Long-Term Corporate Obligation Rating (Domestic); Structured Finance Long-Term Rating (Domestic)',
 'Senior Unsecured (Foreign); Long-Term Corporate Obligation Rating (Foreign); Structured Finance Long-Term Rating (Foreign)',
 'Subordinate (Domestic)',
 'Subordinate (Foreign)'
]

ExtendedSecurityRatingTable = ExtendedSecurityRatingTable[
((ExtendedSecurityRatingTable['agency']=='АКРА') & (ExtendedSecurityRatingTable['official_name'].isin(akra_list))) |
((ExtendedSecurityRatingTable['agency']=='Эксперт РА') & (ExtendedSecurityRatingTable['official_name'].isin(expert_list))) |
((ExtendedSecurityRatingTable['agency']=='НКР') & (ExtendedSecurityRatingTable['official_name'].isin(nkr_list))) |
((ExtendedSecurityRatingTable['agency']=='НРА') & (ExtendedSecurityRatingTable['official_name'].isin(nra_list))) |
((ExtendedSecurityRatingTable['agency']=='Мудис Интерфакс') & (ExtendedSecurityRatingTable['official_name'].isin(moodys_list)))
 ]

ExtendedSecurityRatingTable = ExtendedSecurityRatingTable\
.dropna(subset=['isin'])\
.drop_duplicates(subset=["isin", "agency"])\
.reset_index(drop=True)

In [13]:
ExtendedSecurityRatingTablePivot = ExtendedSecurityRatingTable.drop_duplicates().pivot(index='isin', columns='agency', values='id_value')
ExtendedSecurityRatingTablePivot.columns = pd.MultiIndex.from_tuples([('Эмиссия', i) for i in ExtendedSecurityRatingTablePivot.columns])

In [14]:
TransformedFintoolReferenceData = FintoolReferenceData.copy()[['isincode', 'nickname', 'fintooltype', 'country', 'faceftname', 'fintoolid',
                   'issuername', 'issuercountry', 'issuerinn', 'issueruid',
                   'borrowername', 'borrowercountry', 'borrowerinn', 'borroweruid',
                   'isguaranteed', 'guaranteetype', 'guaranteeamount', 'guarantval',
                   ]].drop_duplicates(subset=['isincode'])
TransformedFintoolReferenceData[['fintoolid', 'issueruid', 'borroweruid']] = TransformedFintoolReferenceData[['fintoolid', 'issueruid', 'borroweruid']].fillna(0).astype('int')
TransformedFintoolReferenceData = TransformedFintoolReferenceData.merge(OfferorsGuarants[['fintoolId','fininstid_guarant']], how='left', left_on='fintoolid', right_on='fintoolId')
ids = list(TransformedFintoolReferenceData.loc[~TransformedFintoolReferenceData['fininstid_guarant'].isna(),'fininstid_guarant'].astype('int'))
TransformedEmitents = Emitents[Emitents.fininstid.isin(ids)][['shortname_rus','country','inn','fininstid']]
TransformedEmitents.columns = ['name_guarant','country_guarant','inn_guarant','fininstid']
TransformedFintoolReferenceData = TransformedFintoolReferenceData.merge(TransformedEmitents, how='left', left_on='fininstid_guarant', right_on='fininstid')
TransformedFintoolReferenceData = TransformedFintoolReferenceData.drop(columns=['fintoolId', 'fininstid'])
TransformedFintoolReferenceData = TransformedFintoolReferenceData.set_index('isincode').rename_axis('ISIN')
TransformedFintoolReferenceData = TransformedFintoolReferenceData.rename(
    columns={
        'nickname': ('Эмиссия','Название'),
        'fintooltype': ('Эмиссия','Тип'),
        'country': ('Эмиссия','Страна'), 
        'faceftname': ('Эмиссия','Валюта'), 
        'fintoolid': ('Эмиссия','Id RuData'),
        'issuername': ('Эмитент','Название'), 
        'issuercountry': ('Эмитент','Страна'), 
        'issuerinn': ('Эмитент','ИНН'), 
        'issueruid': ('Эмитент','Id RuData'), 
        'borrowername': ('Заемщик','Название'),
        'borrowercountry': ('Заемщик','Страна'), 
        'borrowerinn': ('Заемщик','ИНН'), 
        'borroweruid': ('Заемщик','Id RuData'), 
        'isguaranteed': ('Гарант','Есть гарантия'),
        'guaranteetype': ('Гарант','Вид гарантии'), 
        'guaranteeamount': ('Гарант','Полнота гарантии'), 
        'guarantval': ('Гарант','Гарантированная сумма'), 
        'fininstid_guarant': ('Гарант','Id RuData'),
        'name_guarant': ('Гарант','Название'), 
        'country_guarant': ('Гарант','Страна'), 
        'inn_guarant': ('Гарант','ИНН')
    }
)
TransformedFintoolReferenceData.columns = pd.MultiIndex.from_tuples(TransformedFintoolReferenceData.columns)

In [15]:
ExtendedFintoolReferenceData = TransformedFintoolReferenceData.join(ExtendedSecurityRatingTablePivot)
new_cols = ExtendedFintoolReferenceData.columns.reindex(['Эмиссия','Эмитент','Заемщик','Гарант'], level=0)
ExtendedFintoolReferenceData = ExtendedFintoolReferenceData.reindex(columns=new_cols[0])

In [16]:
ExtendedCompanyRatingsTable = CompanyRatingsTable.merge(ListRatings[['id','agency','official_name','term_type_name','scale_type_name','scale_id']], how='left', left_on='id_rating', right_on='id')
##добавляем уровень рейтинг
ExtendedCompanyRatingsTable = ExtendedCompanyRatingsTable.merge(ListScaleValues, how='left', left_on=['scale_id', 'last'], right_on=['scale_id', 'rating_value'])

In [17]:
ExtendedCompanyRatingsTable = ExtendedCompanyRatingsTable[['name_ra', 'dt', 'last', 'last_dt', 'change', 'forecast', 'advanced',
           'prev', 'prev_dt', 'id_rating', 'code', 'code_type', 'company_name',
           'id_x', 'agency', 'official_name', 'term_type_name', 'scale_type_name',
           'scale_id', 'aggregation_level', 'aggregation_type']].rename(columns={'id_x': 'id', 'aggregation_level': 'level', 'aggregation_type': 'agg_type'})
ExtendedCompanyRatingsTable['id_rating'] = ExtendedCompanyRatingsTable['id_rating'].fillna(0)

In [18]:
ExtendedCompanyRatingsTable = ExtendedCompanyRatingsTable[ExtendedCompanyRatingsTable['agency'].isin(['АКРА','Эксперт РА','НКР','НРА','Мудис Интерфакс'])]
ExtendedCompanyRatingsTable = ExtendedCompanyRatingsTable[~ExtendedCompanyRatingsTable['last'].isin(['****','Приостановлен','Снят'])]
ExtendedCompanyRatingsTable = ExtendedCompanyRatingsTable[ExtendedCompanyRatingsTable['term_type_name'] != 'Краткосрочный']

In [19]:
akra_list=[
 'Рейтинг эмитентов',
 'Long-term credit rating',
 'Long-term local currency credit rating',
 'Long-term foreign currency credit rating',
]
expert_list=[
 'Рейтинг кредитоспособности организаций, осуществляющих микрофинансовую деятельность (Республика Казахстан)',
 'Рейтинг кредитоспособности проектных компаний',
 'Рейтинги кредитоспособности банка Республики Беларусь',
 'Рейтинги кредитоспособности банков',
 'Рейтинги кредитоспособности депозитариев',
 'Рейтинги кредитоспособности лизинговых компаний',
 'Рейтинги кредитоспособности микрофинансовых организаций',
 'Рейтинги кредитоспособности нефинансовых компаний',
 'Рейтинги кредитоспособности нефинансовых компаний Республики Беларусь',
 'Рейтинги кредитоспособности нефинансовых компаний Республики Казахстан',
 'Рейтинги кредитоспособности субъектов федерации и муниципальных образований',
 'Рейтинги кредитоспособности факторинговых компаний',
 'Рейтинги кредитоспособности финансовых компаний',
 'Рейтинги кредитоспособности холдинговых компаний',
 'Рейтинги надежности компаний по страхованию жизни',
 'Рейтинги финансовой надежности негосударственных пенсионных фондов',
 'Рейтинги финансовой надежности страховых компаний'
]
nkr_list=[
 'Кредитный рейтинг НКР по Национальной шкале, Банки',
 'Кредитный рейтинг НКР по Национальной шкале, Нефинансовые компании'
]
nra_list=[
 'Кредитный рейтинг кредитных организаций',
 'Кредитный рейтинг нефинансовых компаний, national scale',
]
moodys_list=[
 'BACKED LT Bank Deposits',
 'BACKED LT Bank Deposits (Domestic)',
 'BACKED LT Bank Deposits (Foreign)',
 'BACKED LT Counterparty Risk Rating',
 'BACKED LT Counterparty Risk Rating (Domestic)',
 'BACKED LT Counterparty Risk Rating (Foreign)',
 'BACKED LT Issuer Rating',
 'BACKED LT Issuer Rating (Domestic)',
 'BACKED LT Issuer Rating (Foreign)',
 'BACKED Senior Unsecured (Domestic)',
 'BACKED Senior Unsecured (Foreign)',
 'Issuer Ratings',
 'LT Bank Deposits',
 'LT Bank Deposits (Domestic)',
 'LT Bank Deposits (Foreign)',
 'LT Bank Deposits (kaz)',
 'LT Bank Deposits (ukr)',
 'LT Corporate Family Ratings',
 'LT Corporate Family Ratings (Domestic)',
 'LT Corporate Family Ratings (Foreign)',
 'LT Corporate Family Ratings (ukr)',
 'LT Counterparty Risk Rating (Domestic)',
 'LT Counterparty Risk Rating (Foreign)',
 'LT Insurance Financial Strength',
 'LT Insurance Financial Strength (Domestic)',
 'LT Insurance Financial Strength (Foreign)',
 'LT Issuer Rating',
 'LT Issuer Rating (Domestic)',
 'LT Issuer Rating (Foreign)',
 'Senior Unsecured',
 'Senior Unsecured (Domestic)',
 'Senior Unsecured (Foreign)',
]

ExtendedCompanyRatingsTable = ExtendedCompanyRatingsTable[
((ExtendedCompanyRatingsTable['agency']=='АКРА') & (ExtendedCompanyRatingsTable['official_name'].isin(akra_list))) |
((ExtendedCompanyRatingsTable['agency']=='Эксперт РА') & (ExtendedCompanyRatingsTable['official_name'].isin(expert_list))) |
((ExtendedCompanyRatingsTable['agency']=='НКР') & (ExtendedCompanyRatingsTable['official_name'].isin(nkr_list))) |
((ExtendedCompanyRatingsTable['agency']=='НРА') & (ExtendedCompanyRatingsTable['official_name'].isin(nra_list))) |
((ExtendedCompanyRatingsTable['agency']=='Мудис Интерфакс') & (ExtendedCompanyRatingsTable['official_name'].isin(moodys_list)))
]

In [20]:
ExtendedCompanyRatingsTable = ExtendedCompanyRatingsTable.dropna(subset=['code'])
ExtendedCompanyRatingsTable = ExtendedCompanyRatingsTable.reset_index(drop=True)

In [21]:
for n, g in ExtendedCompanyRatingsTable.groupby(['code', 'agency']):
    if g.shape[0] > 1: 
        try:
            m = g['level'].idxmin(skipna=False)
            if pd.isna(m):
                m = g.index[0]
            to_drop = g.index.drop(m)
            ExtendedCompanyRatingsTable = ExtendedCompanyRatingsTable.drop(index=to_drop)
        except:
            print(g)
            raise
ExtendedCompanyRatingsTable = ExtendedCompanyRatingsTable.dropna(subset=['code'])
ExtendedCompanyRatingsTable = ExtendedCompanyRatingsTable.reset_index(drop=True)
ExtendedCompanyRatingsTable['code'] = ExtendedCompanyRatingsTable['code'].astype(int)

In [22]:
ExtendedCompanyRatingsPivot = ExtendedCompanyRatingsTable.pivot(index='code', columns='agency', values='last')

In [23]:
df1 = ExtendedFintoolReferenceData.copy()
df1.columns =df1.columns.to_flat_index()
for k in ['Эмитент','Заемщик','Гарант']:
    df2 = ExtendedCompanyRatingsPivot.copy()
    df2.columns = [(k, i) for i in df2.columns]
    df1 = df1.merge(df2, how='left', left_on=[(k, 'Id RuData')], right_index=True)
df1.columns = pd.MultiIndex.from_tuples(df1.columns)
new_cols = df1.columns.reindex(['Эмиссия', 'Эмитент', 'Заемщик', 'Гарант'], level=0)
df1 = df1.reindex(columns=new_cols[0])
ExtendedFintoolReferenceData = df1

In [24]:
df = Emitents[Emitents['fininstid'].isin(ExtendedCompanyRatingsPivot.index)]
CompanyRatings = df.merge(ExtendedCompanyRatingsPivot, left_on='fininstid', right_index=True, how='left')

#### Шаг 3. Рейтинги облигаций из фактовой таблицы бумаг FintoolReferenceData. Рейтинги облигаций из фактовой таблицы компаний CompanyRatingsTable

In [25]:
BondRatings = ExtendedFintoolReferenceData['Эмиссия'].reset_index()
id_vars = BondRatings.columns[:6]
value_vars = BondRatings.columns[6:]
meltBonds = pd.melt(BondRatings, id_vars=id_vars, value_vars=value_vars, var_name='agency', value_name='rate')
meltBonds = meltBonds[~meltBonds.rate.isna()].sort_values('Id RuData')
meltBonds = meltBonds[meltBonds["Тип"] == 'Облигация']

In [26]:
CompanyRatings = pd.concat([
    CompanyRatings[["id_emitent", "shortname_rus", "inn", "update_date", "country_oksm", "have_rating", "have_risk", "tin", "fininstid", "reg_code", "lei_code"]]
    , CompanyRatings.iloc[:, CompanyRatings.columns.get_loc('rn')+1:]
], axis=1)
id_vars = CompanyRatings.columns[:11]
value_vars = CompanyRatings.columns[11:]
meltCompanies = pd.melt(CompanyRatings, id_vars=id_vars, value_vars=value_vars, var_name='agency', value_name='rate')
meltCompanies = meltCompanies[meltCompanies['rate'].notnull()].sort_values('id_emitent')
meltCompanies['inn'] = np.where(meltCompanies['country_oksm'] == 643, meltCompanies['inn'], meltCompanies['tin'] )
meltCompanies['country_oksm'] = meltCompanies['country_oksm'].astype(np.int32)
meltCompanies['type'] = "ЮЛ"

In [27]:
meltBonds.rename(columns={
    "Id RuData": "id_emitent",
    "Название": "company_name", 
    "Тип": "rate_type",
    "ISIN": "inn/isin",
    "Страна": "country"
    }, inplace=True)
meltCompanies.rename(columns={
    "id_rudata": "id_emitent",
    "shortname_rus": "company_name", 
    "type": "rate_type",
    "inn": "inn/isin",
    "country_oksm": "country_code"
    }, inplace=True)
meltBonds["fininstid"] = np.nan
meltBonds["reg_code"] = np.nan
meltBonds["lei_code"] = np.nan
meltBonds["report_date"] = pd.to_datetime(REPORT_DATE)
meltCompanies["report_date"] = pd.to_datetime(REPORT_DATE)

Справочник кодов стран ОКСМ

In [28]:
countries = pd.read_sql(
    """
    SELECT DISTINCT country, alpha2
    FROM countries
    """
    , RuDataDF.engine)
countries["country"] = countries["country"].astype(np.int32)
co = countries.set_index("alpha2").country.to_dict()
meltBonds["country_code"] = meltBonds["country"].map(co)

#### Шаг 4. Получаем результирующий результат по рейтингам облигаций и компаний и экспорт в excel

In [1]:
meltBonds = meltBonds[["id_emitent", "fininstid", "company_name", "inn/isin", "country_code", "agency", "rate", "report_date", "rate_type", "reg_code", "lei_code"]]
meltCompanies = meltCompanies[["id_emitent", "fininstid", "company_name", "inn/isin", "country_code", "agency", "rate", "report_date", "rate_type", "reg_code", "lei_code"]]
ResultRatings = pd.concat([meltBonds, meltCompanies], axis=0)
with pd.ExcelWriter('data/Output/ratings.xlsx', datetime_format='dd.mm.yyyy') as writer:
    ResultRatings.to_excel(writer, index=False)

NameError: name 'meltBonds' is not defined

In [1]:
from functions.send_email import main
main("ratings.xlsx")